<a href="https://colab.research.google.com/github/johanhoffman/DD2363-VT19/blob/bozzato/Lab-6/bozzato_lab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 6: Partial differential equation**
**Bozzato Federico**

# **Abstract**

Short summary of the lab report. State the objectives, methods used, main results and conlusions. 

#**About the code**

**Author:** Federico Bozzato

In [1]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2019 Johan Hoffman (jhoffman@kth.se)

# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them. 

In [0]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np
from numpy import linalg as la
from matplotlib import pyplot as plt


# **Introduction**

Give a short description of the problem investigated in the report, and provide some background information so that the reader can understand the context. 

Briefly describe what method you have chosen to solve the problem, and justify why you selected that method. 

Here you can express mathematics through Latex syntax, and use hyperlinks for references.

[Hyperlink to DD2363 course website.](https://kth.instructure.com/courses/7500)

$
{\displaystyle \frac{\partial u}{\partial t}} + u\cdot \nabla u +\nabla p = f, \quad \nabla \cdot u=0$



#Methods

##Mandatory assignment

###1. Function: FEM for 1D boundary value problem $-u''(x) = f(x)$ with $u(0)=u(1)=0$

In [0]:
def bvp1D(fun, mesh):
  
  ##############################
  ###### STIFFNESS MATRIX ######
  ##############################
  def stiffnessAssembler(mesh):
    n= len(mesh)
    S= np.zeros((n,n))
    
    for i in range(0,n-1):
      h= mesh[i+1] - mesh[i]
      S[i,i]= S[i,i] + 1/h
      S[i,i+1]= S[i,i+1] - 1/h
      S[i+1,i]= S[i+1,i] - 1/h
      S[i+1,i+1]= S[i+1,i+1] + 1/h

    return S
  
  ##############################
  ######  LOAD ASSEMBLER  ######
  ##############################
  def loadAssembler(fun, mesh):    # quadrature b with trapezoidal rule
    n= len(mesh)
    b= np.zeros((n,1))
    for i in range(0,n-1):
      h= mesh[i+1] - mesh[i]
      b[i]= b[i] + fun(mesh[i]) * h/2
      b[i+1]= b[i+1] + fun(mesh[i+1]) * h/2
    
    return b
  
  S= stiffnessAssembler(mesh)
  b= loadAssembler(fun, mesh)
  alpha= la.lstsq(S,b, rcond=None)[0]
  
  return S, b, alpha

In [16]:
def fun(u):
  return -4*u

mesh= [0, 3, 3.5, 5]

S, b, alpha=bvp1D(fun, mesh)

print('S=\n',S)
print('b=\n',b)
print('x=\n',alpha)

S=
 [[ 0.33333333 -0.33333333  0.          0.        ]
 [-0.33333333  2.33333333 -2.          0.        ]
 [ 0.         -2.          2.66666667 -0.66666667]
 [ 0.          0.         -0.66666667  0.66666667]]
b=
 [[  0.]
 [-21.]
 [-14.]
 [-15.]]
x=
 [[ 30.0625]
 [ -7.4375]
 [ -9.4375]
 [-13.1875]]


##Extra assignment

###2. Function: FEM for 2D Poisson equation -Delta u(x) = f(x), with triangular mesh

# **Results**

Present the results. If the result is an algorithm that you have described under the *Methods* section, you can present the data from verification and performance tests in this section. If the result is the output from a computational experiment this is where you present a selection of that data. 

# **Discussion**

Summarize your results and your conclusions. Were the results expected or surprising. Do your results have implications outside the particular problem investigated in this report? 

# **APPENDIX**

[fast stiffness](https://www.hindawi.com/journals/jam/2014/932314/)